In [1]:
import re

from pathlib import Path
from num2words import num2words

In [2]:
transcripts_path = Path('/home/dino/datasets/automin-2021-confidential-data/task-A-elitr-minuting-corpus-en/train')
anootations_pattern = 'transcript_*.deidentified.txt'

In [3]:
transcripts_paths = []
for path in Path(transcripts_path).rglob(anootations_pattern):
    transcripts_paths += [path]
transcripts_paths = sorted(transcripts_paths)

In [4]:
transcripts_paths[:2]

[PosixPath('/home/dino/datasets/automin-2021-confidential-data/task-A-elitr-minuting-corpus-en/train/meeting_en_train_001/transcript_MAN2_annot02.deidentified.txt'),
 PosixPath('/home/dino/datasets/automin-2021-confidential-data/task-A-elitr-minuting-corpus-en/train/meeting_en_train_002/transcript_MAN_annot17.deidentified.txt')]

In [5]:
def clean_text(text):
    text = text.lower()
    text = text.replace(u'\xa0', u' ')
    text = re.sub('[[(<]\S+[])>]', '', text)
    match = re.search('(?<=\s)[0-9]+(th|rd|nd)', text)
    while match !=  None:
        offset = match.start(0)
        onset = match.end(0)
        text = text[:offset] + \
        num2words(int(text[offset:onset-2]), to='ordinal') + \
        text[onset:]
        match = re.search('(?<=\s)[0-9]+(th|rd|nd)', text)
    match = re.search('(?<=\s)[0-9]+(?=[a-z\s])', text)
    while match !=  None:
        offset = match.start(0)
        onset = match.end(0)
        text = text[:offset] + \
        num2words(int(text[offset:onset])) + ' ' + \
        text[onset:]
        match = re.search('(?<=\s)[0-9]+(?=[a-z\s])', text)
    text = re.sub('[.!?]', '\n', text)
    text = re.sub('[^qwertyuiopasdfghjklzxcvbnm\-\'\s]', '', text)
    text = re.sub('(?<!\w)-(?!\w)', '', text)
    text = re.sub('(?<!\w)-(?=\w)', '', text)
    text = re.sub('(?<=\w)-(?!\w)', '', text)
    text = re.sub('\s(uh|uhm|ehm)\s', ' ', text)
    text = re.sub(' \'m', '\'m', text)
    text = re.sub('[ ]*\n+[ ]*', '\n', text)
    text = re.sub('[ ]+', ' ', text)
    text = re.sub('^[ \-]+', '', text)
    text = re.sub('[ \-]+$', '', text)
    lines = [line for line in text.split('\n') if line != '']
    return lines

In [6]:
texts = []
for text_file in transcripts_paths:
    with open(text_file) as f:
        texts += clean_text(f.read())

<ipython-input-5-24d96a0b7262>:4: FutureWarning: Possible nested set at position 1
  text = re.sub('[[(<]\S+[])>]', '', text)


In [7]:
with open(transcripts_path / 'automin.corpus', 'w') as f:
    f.write('\n'.join(texts))